In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## 超参数

In [2]:
input_size = 784
num_class = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

## MNIST 数据集（图像和标签）

In [3]:
train_dataset = torchvision.datasets.MNIST(root='../../data/MNIST',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=False)

In [5]:
test_dataset = torchvision.datasets.MNIST(root='../../data/MNIST',
                                        train=False,
                                        transform=transforms.ToTensor(),
                                        download=False)

## 数据生成器Data loader（input pipeline）

In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

## Logistic回归模型

In [8]:
model = nn.Linear(input_size, num_class).cuda()

In [9]:
# 损失 和 优化器
criterion = nn.CrossEntropyLoss().cuda() # nn.CrossEntropyLoss() 在内部计算 softmax
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
# 模型训练
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 将图像改成(batch_size, input_size)
        images = images.reshape(-1, 28*28).cuda()
        labels = labels.cuda()
        #前向传播
        outputs = model(images).cuda()
        loss = criterion(outputs, labels).cuda()
        
        #反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1) %100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_steps, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2258
Epoch [1/5], Step [200/600], Loss: 2.1483
Epoch [1/5], Step [300/600], Loss: 2.0908
Epoch [1/5], Step [400/600], Loss: 1.9965
Epoch [1/5], Step [500/600], Loss: 1.9328
Epoch [1/5], Step [600/600], Loss: 1.9064
Epoch [2/5], Step [100/600], Loss: 1.6969
Epoch [2/5], Step [200/600], Loss: 1.7302
Epoch [2/5], Step [300/600], Loss: 1.7412
Epoch [2/5], Step [400/600], Loss: 1.6415
Epoch [2/5], Step [500/600], Loss: 1.6076
Epoch [2/5], Step [600/600], Loss: 1.6110
Epoch [3/5], Step [100/600], Loss: 1.3619
Epoch [3/5], Step [200/600], Loss: 1.4610
Epoch [3/5], Step [300/600], Loss: 1.5090
Epoch [3/5], Step [400/600], Loss: 1.4091
Epoch [3/5], Step [500/600], Loss: 1.3926
Epoch [3/5], Step [600/600], Loss: 1.4082
Epoch [4/5], Step [100/600], Loss: 1.1426
Epoch [4/5], Step [200/600], Loss: 1.2798
Epoch [4/5], Step [300/600], Loss: 1.3518
Epoch [4/5], Step [400/600], Loss: 1.2507
Epoch [4/5], Step [500/600], Loss: 1.2438
Epoch [4/5], Step [600/600], Loss:

## 测试模型

In [11]:
#在测试阶段，我们不用计算梯度（为了内存利用效率）
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).cuda()
        labels = labels.cuda()
        outputs = model(images).cuda()
        _, predicted = torch.max(outputs.data, 1)# torch.max返回最大的元素以及索引
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
    print('Accuracy of the model on the 10000 test images: {}%'.format(100 * correct / total))
        

Accuracy of the model on the 10000 test images: 82%


## 保存模型检查点

In [12]:
torch.save(model.state_dict(), 'model.ckpt')